# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [27]:
import numpy as np
import problem_unittests as tests
from string import punctuation
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # remove the punctuation
#    all_text = ''.join([c for c in text if c not in punctuation])
    
    # remove the \n
#    Processed_text = all_text.split('\n')
    # creating the set words 
#    all_text = ' '.join(Processed_text)
#    all_text = ' '.join(text)
#    words = all_text.split()

    # Create dictionary that maps words to integers
    # Number of times the word has been used
    counts = Counter(text)
    Vocab = sorted(counts, key=counts.get, reverse=True)

    # dic of int_to_vocab
    vocab_to_int = dict(tuple([(v, ii) for ii, v in enumerate(Vocab, 1)]))
    
    # dic of vocab_to_int
    int_to_vocab = dict(tuple([(v, ii) for ii, v in vocab_to_int.items()]))
    
    print(vocab_to_int)
    # TODO: Implement Function
    return vocab_to_int, int_to_vocab,

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'moe_szyslak': 1, 'mike': 2, 'rotch': 3, 'you': 4, 'your': 5, 'to': 6, 'drink': 7, 'the': 8, 'yeah': 9, 'name': 10, 'on': 11, 'hey': 12, 'one': 13, "i'm": 14, 'gonna': 15, 'my': 16, 'homer': 17, 'not': 18, 'problems': 19, 'should': 20, "moe's": 21, 'tavern': 22, 'where': 23, 'elite': 24, 'meet': 25, 'bart_simpson': 26, 'eh': 27, 'hello': 28, 'is': 29, 'there': 30, 'last': 31, 'hold': 32, "i'll": 33, 'check': 34, 'has': 35, 'anybody': 36, 'seen': 37, 'lately': 38, 'listen': 39, 'little': 40, 'puke': 41, 'of': 42, 'these': 43, 'days': 44, 'catch': 45, 'and': 46, 'carve': 47, 'back': 48, 'with': 49, 'an': 50, 'ice': 51, 'pick': 52, 'whats': 53, 'matter': 54, "you're": 55, 'normal': 56, 'effervescent': 57, 'self': 58, 'homer_simpson': 59, 'i': 60, 'got': 61, 'moe': 62, 'give': 63, 'me': 64, 'another': 65, 'forget': 66, 'barney_gumble': 67, 'only': 68, 'enhance': 69, 'social': 70, 'skills': 71}
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    Token_Punctuation = dict([('.','||Period||'), (',','||Comma||'), ('"','||QuotationMark||'), 
                         ('?','||QuestionMark||'), ('(','||LeftParentheses||'), (')','||RightParentheses||'), 
                         ('--','||Dash||'), ('\n','||return||'), ('!', '||EmulationPoint||'), (';', '||Semicolon||')])

    # TODO: Implement Function
    return Token_Punctuation

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)


Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1


C:\Users\Ehsan\Anaconda3\envs\TV-Scripts\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    Inputs = tf.placeholder(tf.int32, (None, None), name='input')
    Targets = tf.placeholder(tf.int32, (None, None), name='Targets')
    LearningRate = tf.placeholder(tf.float32, None, name = 'LearningRate')
    # TODO: Implement Function
    return Inputs, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # Basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    # Stack up multiple LSTM layers, for deep learning
    Cell = tf.contrib.rnn.MultiRNNCell([lstm])
    
    # Getting an initial state of all zeros
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(InitialState, name='initial_state')
    
    # TODO: Implement Function
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [32]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embedded = tf.nn.embedding_lookup(embedding, input_data)

    # TODO: Implement Function
    return embedded


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name="final_state")
    
    # TODO: Implement Function
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embedding = get_embed(input_data, vocab_size, embed_dim)
    outputs, FinalState = build_rnn(cell, embedding)
    Logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    # TODO: Implement Function
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_batch = len(int_text) // (batch_size * seq_length)
    
    # Remove end of the data set and roll back the target data
    x_data = np.array(int_text[:seq_length * batch_size * n_batch])
    y_data = np.roll(x_data,-1)
    
    # reshape the X_data and y_data into x_batch and y_batch 
    x_batches = np.split(x_data.reshape(batch_size, -1), n_batch, 1)

    y_batches = np.split(y_data.reshape(batch_size, -1), n_batch, 1)
    
    # zipping them all in one batch array 
    batch = np.array(list(zip(x_batches, y_batches)))
    
    # TODO: Implement Function
    return batch


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [76]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 120
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.005
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [77]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab) + 1
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [78]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/53   train_loss = 8.820
Epoch   0 Batch    5/53   train_loss = 7.689
Epoch   0 Batch   10/53   train_loss = 6.459
Epoch   0 Batch   15/53   train_loss = 6.750
Epoch   0 Batch   20/53   train_loss = 6.712
Epoch   0 Batch   25/53   train_loss = 6.453
Epoch   0 Batch   30/53   train_loss = 6.363
Epoch   0 Batch   35/53   train_loss = 6.150
Epoch   0 Batch   40/53   train_loss = 6.087
Epoch   0 Batch   45/53   train_loss = 5.698
Epoch   0 Batch   50/53   train_loss = 5.859
Epoch   1 Batch    2/53   train_loss = 5.637
Epoch   1 Batch    7/53   train_loss = 5.714
Epoch   1 Batch   12/53   train_loss = 5.506
Epoch   1 Batch   17/53   train_loss = 5.544
Epoch   1 Batch   22/53   train_loss = 5.352
Epoch   1 Batch   27/53   train_loss = 5.456
Epoch   1 Batch   32/53   train_loss = 5.432
Epoch   1 Batch   37/53   train_loss = 5.346
Epoch   1 Batch   42/53   train_loss = 5.255
Epoch   1 Batch   47/53   train_loss = 5.206
Epoch   1 Batch   52/53   train_loss = 5.136
Epoch   2 

Epoch  17 Batch   14/53   train_loss = 2.795
Epoch  17 Batch   19/53   train_loss = 2.801
Epoch  17 Batch   24/53   train_loss = 2.810
Epoch  17 Batch   29/53   train_loss = 2.816
Epoch  17 Batch   34/53   train_loss = 2.757
Epoch  17 Batch   39/53   train_loss = 2.723
Epoch  17 Batch   44/53   train_loss = 2.792
Epoch  17 Batch   49/53   train_loss = 2.800
Epoch  18 Batch    1/53   train_loss = 2.632
Epoch  18 Batch    6/53   train_loss = 2.719
Epoch  18 Batch   11/53   train_loss = 2.780
Epoch  18 Batch   16/53   train_loss = 2.809
Epoch  18 Batch   21/53   train_loss = 2.598
Epoch  18 Batch   26/53   train_loss = 2.744
Epoch  18 Batch   31/53   train_loss = 2.690
Epoch  18 Batch   36/53   train_loss = 2.709
Epoch  18 Batch   41/53   train_loss = 2.657
Epoch  18 Batch   46/53   train_loss = 2.745
Epoch  18 Batch   51/53   train_loss = 2.761
Epoch  19 Batch    3/53   train_loss = 2.679
Epoch  19 Batch    8/53   train_loss = 2.642
Epoch  19 Batch   13/53   train_loss = 2.767
Epoch  19 

Epoch  34 Batch   28/53   train_loss = 1.816
Epoch  34 Batch   33/53   train_loss = 1.903
Epoch  34 Batch   38/53   train_loss = 1.898
Epoch  34 Batch   43/53   train_loss = 1.819
Epoch  34 Batch   48/53   train_loss = 1.819
Epoch  35 Batch    0/53   train_loss = 1.761
Epoch  35 Batch    5/53   train_loss = 1.886
Epoch  35 Batch   10/53   train_loss = 1.866
Epoch  35 Batch   15/53   train_loss = 1.807
Epoch  35 Batch   20/53   train_loss = 1.890
Epoch  35 Batch   25/53   train_loss = 1.891
Epoch  35 Batch   30/53   train_loss = 1.738
Epoch  35 Batch   35/53   train_loss = 1.833
Epoch  35 Batch   40/53   train_loss = 1.772
Epoch  35 Batch   45/53   train_loss = 1.846
Epoch  35 Batch   50/53   train_loss = 1.802
Epoch  36 Batch    2/53   train_loss = 1.707
Epoch  36 Batch    7/53   train_loss = 1.826
Epoch  36 Batch   12/53   train_loss = 1.819
Epoch  36 Batch   17/53   train_loss = 1.682
Epoch  36 Batch   22/53   train_loss = 1.884
Epoch  36 Batch   27/53   train_loss = 1.741
Epoch  36 

Epoch  51 Batch   42/53   train_loss = 1.299
Epoch  51 Batch   47/53   train_loss = 1.350
Epoch  51 Batch   52/53   train_loss = 1.406
Epoch  52 Batch    4/53   train_loss = 1.381
Epoch  52 Batch    9/53   train_loss = 1.265
Epoch  52 Batch   14/53   train_loss = 1.193
Epoch  52 Batch   19/53   train_loss = 1.336
Epoch  52 Batch   24/53   train_loss = 1.268
Epoch  52 Batch   29/53   train_loss = 1.286
Epoch  52 Batch   34/53   train_loss = 1.288
Epoch  52 Batch   39/53   train_loss = 1.224
Epoch  52 Batch   44/53   train_loss = 1.353
Epoch  52 Batch   49/53   train_loss = 1.328
Epoch  53 Batch    1/53   train_loss = 1.264
Epoch  53 Batch    6/53   train_loss = 1.318
Epoch  53 Batch   11/53   train_loss = 1.358
Epoch  53 Batch   16/53   train_loss = 1.344
Epoch  53 Batch   21/53   train_loss = 1.114
Epoch  53 Batch   26/53   train_loss = 1.229
Epoch  53 Batch   31/53   train_loss = 1.287
Epoch  53 Batch   36/53   train_loss = 1.327
Epoch  53 Batch   41/53   train_loss = 1.229
Epoch  53 

Epoch  69 Batch    3/53   train_loss = 0.969
Epoch  69 Batch    8/53   train_loss = 1.008
Epoch  69 Batch   13/53   train_loss = 1.003
Epoch  69 Batch   18/53   train_loss = 1.016
Epoch  69 Batch   23/53   train_loss = 1.020
Epoch  69 Batch   28/53   train_loss = 1.044
Epoch  69 Batch   33/53   train_loss = 1.094
Epoch  69 Batch   38/53   train_loss = 1.056
Epoch  69 Batch   43/53   train_loss = 0.962
Epoch  69 Batch   48/53   train_loss = 1.012
Epoch  70 Batch    0/53   train_loss = 1.018
Epoch  70 Batch    5/53   train_loss = 1.063
Epoch  70 Batch   10/53   train_loss = 1.036
Epoch  70 Batch   15/53   train_loss = 1.007
Epoch  70 Batch   20/53   train_loss = 1.025
Epoch  70 Batch   25/53   train_loss = 1.010
Epoch  70 Batch   30/53   train_loss = 0.966
Epoch  70 Batch   35/53   train_loss = 0.999
Epoch  70 Batch   40/53   train_loss = 0.985
Epoch  70 Batch   45/53   train_loss = 1.070
Epoch  70 Batch   50/53   train_loss = 1.006
Epoch  71 Batch    2/53   train_loss = 0.921
Epoch  71 

Epoch  86 Batch   17/53   train_loss = 0.844
Epoch  86 Batch   22/53   train_loss = 0.958
Epoch  86 Batch   27/53   train_loss = 0.852
Epoch  86 Batch   32/53   train_loss = 0.801
Epoch  86 Batch   37/53   train_loss = 0.854
Epoch  86 Batch   42/53   train_loss = 0.881
Epoch  86 Batch   47/53   train_loss = 0.886
Epoch  86 Batch   52/53   train_loss = 0.948
Epoch  87 Batch    4/53   train_loss = 0.876
Epoch  87 Batch    9/53   train_loss = 0.803
Epoch  87 Batch   14/53   train_loss = 0.800
Epoch  87 Batch   19/53   train_loss = 0.872
Epoch  87 Batch   24/53   train_loss = 0.853
Epoch  87 Batch   29/53   train_loss = 0.839
Epoch  87 Batch   34/53   train_loss = 0.803
Epoch  87 Batch   39/53   train_loss = 0.823
Epoch  87 Batch   44/53   train_loss = 0.929
Epoch  87 Batch   49/53   train_loss = 0.875
Epoch  88 Batch    1/53   train_loss = 0.814
Epoch  88 Batch    6/53   train_loss = 0.840
Epoch  88 Batch   11/53   train_loss = 0.899
Epoch  88 Batch   16/53   train_loss = 0.930
Epoch  88 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [79]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [80]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [81]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    # TODO: Implement Function
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [131]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return int_to_vocab[probabilities.argmax(axis=0)]
#    return int_to_vocab[np.argmax(probabilities)]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [132]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(delighted) ooh, a text.(checks pockets) check that poor girls age!
carl_carlson: what are you all lookin' at the theater next to the bar.
moe_szyslak: oh, what are you doing?
moe_szyslak:(uneasy) oh, here comes.
homer_simpson:(finishing with romantic.) you know what i think you're awesome.
homer_simpson:(admitting) just keep your daughter to a woman of my new microbrew.
lenny_leonard: it's like, you gotta do it fast.
moe_szyslak: i don't know what i think so.
kirk_van_houten: you know, i do, but i have been there? a pretzel? wow, looks like i can't talk, it's groveling that works. eh, we've got a job!
moe_szyslak:(to patrons) figure of you, and it say the wrong thing...
young_marge: tonight!
homer_simpson:(surprised) really actually, uh, horrible addiction compels you without the fork in here.


moe_szyslak: hey, what's wrong problem. you gotta hit it


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.